In [8]:
import pandas as pd
from binance.spot import Spot
from tqdm.auto import tqdm
import numpy as np  
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime, timedelta

In [9]:
client = Spot()

In [10]:
def processing_data(data):
    data.columns = ['kline_open_time', 'open_price','high_price','low_price','close_price']
    data['kline_open_time'] = pd.to_datetime(data['kline_open_time'],unit='ms')
    data[['open_price','high_price','low_price','close_price']] = data[['open_price','high_price','low_price','close_price']].astype(float)
    data.sort_values('kline_open_time', inplace=True)
    data.reset_index(drop=True,inplace = True)
    

In [11]:
# Соберём данные 3000 часовых свечей
# Т.к. библиотека binance API позволяет получить максимум 1000 свечей за один запрост, 
# можем написать цыкл 
res = pd.DataFrame()
v = 0

for i in tqdm(range(3)):
    a = client.klines("BTCUSDT", '1h', limit = 1000, endTime = client.time()['serverTime']- v*i)
    res = pd.concat([res, pd.DataFrame(a).iloc[:,:5]])
    v = 3600000000 +3600000


  0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
processing_data(res)
res


,kline_open_time,open_price,high_price,low_price,close_price
0,2022-12-23 22:00:00,16812.40,16812.40,16765.67,16792.92
1,2022-12-23 23:00:00,16792.92,16799.27,16772.44,16778.50
2,2022-12-24 00:00:00,16778.52,16816.62,16776.62,16805.92
3,2022-12-24 01:00:00,16806.24,16827.00,16798.03,16814.05
4,2022-12-24 02:00:00,16813.67,16831.00,16801.29,16828.14
...,...,...,...,...,...
2995,2023-04-27 19:00:00,29772.12,29858.19,29450.98,29673.98
2996,2023-04-27 20:00:00,29673.98,29754.99,29467.99,29619.30
2997,2023-04-27 21:00:00,29619.30,29783.15,29536.36,29647.07
2998,2023-04-27 22:00:00,29647.08,29664.01,29255.53,29385.20


In [13]:
fig = px.line(res, x="kline_open_time", y="close_price")
fig.show()

In [14]:
res.to_csv('./train_data.csv', index=False)